In [62]:
from prepare_language_folder import prepare_language_folder
from preprocessing.data_collection.multipleye_data_collection import MultipleyeDataCollection
from pathlib import Path

In [ ]:
# import os
# print(os.environ["PATH"])

In [63]:
data_collection_name = 'MultiplEYE_SQ_CH_Zurich_1_2025'

If necessary, prepare the data folder by unzipping the downloaded files. Works only for MultiplEYE and MeRID data collections so far. Also, there might be some manual steps necessary.

In [64]:
prepare_language_folder(data_collection_name)

No zipped data found for test_sessions. Proceeding with existing data.
No zipped data found for 013_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for .DS_Store. Proceeding with existing data.
No zipped data found for 006_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for 012_SQ_CH_1_ET1_start_after_trial_5. Proceeding with existing data.
No zipped data found for 010_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for 009_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for data_piloting_stimuli_MultiplEYE_SQ_CH_Zurich_1_2025participant_id_1_to_5. Proceeding with existing data.
No zipped data found for 007_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for NotFoundError. Proceeding with existing data.
No zipped data found for 012_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for pilot_sessions. Proceeding with existing data.
No zipped data found for 008_SQ_CH_1_ET1. Proceeding with

In [65]:
this_repo = Path().resolve()
data_folder_path = this_repo / "data" / data_collection_name

In [66]:
multipleye_sq = MultipleyeDataCollection.create_from_data_folder(data_folder_path)

Folder test_sessions does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.
Folder data_piloting_stimuli_MultiplEYE_SQ_CH_Zurich_1_2025participant_id_1_to_5 does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.
Folder pilot_sessions does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.


Converting EDF to ASC: 100%|██████████| 9/9 [00:00<00:00, 8780.82it/s]


Parsing participant data : 012_SQ_CH_1_ET1_start_after_trial_5:   0%|          | 0/9 [00:00<?, ?it/s]WARNING:root:Session 012_SQ_CH_1_ET1_start_after_trial_5 has been restarted.
Parsing participant data : 011_SQ_CH_1_ET1: 100%|██████████| 9/9 [00:00<00:00, 441.45it/s]


In [67]:
sessions = [s for s in multipleye_sq]

In [68]:
sessions[0]

{'session_folder_path': '/Users/anastassiashaitarova/Documents/postdoc-life/openEye/multipleye-preprocessing/data/MultiplEYE_SQ_CH_Zurich_1_2025/eye-tracking-sessions/006_SQ_CH_1_ET1',
 'session_file_path': PosixPath('/Users/anastassiashaitarova/Documents/postdoc-life/openEye/multipleye-preprocessing/data/MultiplEYE_SQ_CH_Zurich_1_2025/eye-tracking-sessions/006_SQ_CH_1_ET1/006sqch1.edf'),
 'session_file_name': '006sqch1.edf',
 'session_folder_name': '006_SQ_CH_1_ET1',
 'session_stimuli': '',
 'is_pilot': False,
 'asc_path': PosixPath('/Users/anastassiashaitarova/Documents/postdoc-life/openEye/multipleye-preprocessing/data/MultiplEYE_SQ_CH_Zurich_1_2025/eye-tracking-sessions/006_SQ_CH_1_ET1/006sqch1.asc'),
 'gaze_path': PosixPath('/Users/anastassiashaitarova/Documents/postdoc-life/openEye/multipleye-preprocessing/data/MultiplEYE_SQ_CH_Zurich_1_2025/quality_reports/006_SQ_CH_1_ET1/006_SQ_CH_1_ET1_gaze.pkl'),
 'completed_stimuli_ids': [13, 7, 2, 8, 10, 6, 9, 12, 3, 4, 1, 11],
 'stimuli_tr

In [69]:
# unpickle gaze data
import pickle
with open(sessions[0]['gaze_path'], 'rb') as handle:
    gaze = pickle.load(handle)

gaze

time,pupil,activity,practice,trial,stimulus,screen,session,pixel,position,velocity
i64,f64,str,bool,str,str,str,str,list[f64],list[f64],list[f64]
2629468,1085.0,"""reading""",true,"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""006_SQ_CH_1_ET1""","[46.3, 126.2]","[-15.974782, -9.88542]","[-1.204677, -0.592322]"
2629469,1093.0,"""reading""",true,"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""006_SQ_CH_1_ET1""","[44.7, 124.9]","[-16.01472, -9.919118]","[-1.191805, -0.608509]"
2629470,1083.0,"""reading""",true,"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""006_SQ_CH_1_ET1""","[44.1, 126.3]","[-16.029693, -9.882828]","[-1.215066, -0.604523]"
2629471,1088.0,"""reading""",true,"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""006_SQ_CH_1_ET1""","[44.7, 126.4]","[-16.01472, -9.880235]","[-1.261583, -0.63971]"
2629472,1087.0,"""reading""",true,"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""006_SQ_CH_1_ET1""","[44.3, 124.7]","[-16.024702, -9.924302]","[-1.251194, -0.60687]"
…,…,…,…,…,…,…,…,…,…,…
10558289,877.0,"""question""",false,"""trial_10""","""Arg_PISARapaNui_11""","""question_11131""","""006_SQ_CH_1_ET1""","[278.9, 887.1]","[-10.015618, 10.229845]","[-0.553149, -0.989099]"
10558290,880.0,"""question""",false,"""trial_10""","""Arg_PISARapaNui_11""","""question_11131""","""006_SQ_CH_1_ET1""","[279.7, 887.1]","[-9.99466, 10.229845]","[-0.434833, -0.882126]"
10558291,878.0,"""question""",false,"""trial_10""","""Arg_PISARapaNui_11""","""question_11131""","""006_SQ_CH_1_ET1""","[278.1, 888.0]","[-10.036573, 10.253125]","[-0.353749, -0.792705]"


In [ ]:
#  This is unnecessary
# multipleye_sq.preprocess_eye_tracking_data()
# sessions_processed = [s for s in multipleye_sq]
# with open(sessions_processed[0]['gaze_path'], 'rb') as handle:
#     gaze_processed = pickle.load(handle)

# gaze_processed

# gazeData = MultipleyeDataCollection.create_gaze_frame("006_SQ_CH_1_ET1")

In [ ]:
from pymovements.gaze.io import from_asc

In [ ]:
def load_session_gaze(session):
    gaze = from_asc(
        file=session['asc_path'],
        events=True
    )
    return gaze

gaze = load_session_gaze(sessions[0])

# Set screen parameters for degrees of visual angle conversion
# a must-have in order to convert pixels to degrees of visual angle
gaze.experiment.screen.distance_cm = 60 
gaze.experiment.screen.height_cm = 28
gaze.experiment.screen.width_cm = 37

In [ ]:
# This gaze has parsed events (recorded by the eye tracker during acquisition),not recomputed from samples by PyMovements
gaze

time,pupil,pixel
i64,f64,list[f64]
2626911,996.0,"[50.7, 122.8]"
2626912,996.0,"[50.4, 124.0]"
2626913,998.0,"[50.8, 123.3]"
2626914,995.0,"[51.8, 123.0]"
2626915,995.0,"[51.6, 123.1]"
…,…,…
10558292,879.0,"[278.5, 885.7]"
10558293,880.0,"[278.7, 886.1]"
10558294,874.0,"[277.6, 885.6]"


In [94]:
gaze.pix2deg()
gaze.pos2vel('smooth')

In [95]:
gaze

time,pupil,pixel,position,velocity
i64,f64,list[f64],list[f64],list[f64]
2626911,996.0,"[50.7, 122.8]","[-15.865846, -9.976727]","[null, null]"
2626912,996.0,"[50.4, 124.0]","[-15.873338, -9.945663]","[null, null]"
2626913,998.0,"[50.8, 123.3]","[-15.863349, -9.963784]","[9.574565, -3.020216]"
2626914,995.0,"[51.8, 123.0]","[-15.838371, -9.97155]","[4.57903, 0.431618]"
2626915,995.0,"[51.6, 123.1]","[-15.843367, -9.968961]","[-12.486222, 10.786545]"
…,…,…,…,…
10558292,879.0,"[278.5, 885.7]","[-10.031679, 10.170738]","[-6.543484, -14.647334]"
10558293,880.0,"[278.7, 886.1]","[-10.026444, 10.181079]","[-4.36183, -12.924581]"
10558294,874.0,"[277.6, 885.6]","[-10.055232, 10.168153]","[5.235921, 4.738033]"


In [ ]:
gaze.detect(method="idt", dispersion_threshold=100, clear=True)

ValueError: interval between timesteps must be constant

In [ ]:
# recompute events (e.g. with IVT), produces fixations
gaze.detect(method="ivt", velocity_threshold=30)

In [42]:
gaze

time,pupil,pixel,position,velocity
i64,f64,list[f64],list[f64],list[f64]
2626911,996.0,"[50.7, 122.8]","[-15.865846, -9.976727]","[null, null]"
2626912,996.0,"[50.4, 124.0]","[-15.873338, -9.945663]","[null, null]"
2626913,998.0,"[50.8, 123.3]","[-15.863349, -9.963784]","[9.574565, -3.020216]"
2626914,995.0,"[51.8, 123.0]","[-15.838371, -9.97155]","[4.57903, 0.431618]"
2626915,995.0,"[51.6, 123.1]","[-15.843367, -9.968961]","[-12.486222, 10.786545]"
…,…,…,…,…
10558292,879.0,"[278.5, 885.7]","[-10.031679, 10.170738]","[-6.543484, -14.647334]"
10558293,880.0,"[278.7, 886.1]","[-10.026444, 10.181079]","[-4.36183, -12.924581]"
10558294,874.0,"[277.6, 885.6]","[-10.055232, 10.168153]","[5.235921, 4.738033]"


In [44]:
import polars as pl

fixations_ivt = gaze.events.frame.filter(pl.col("name") == "fixation")
fixations_ivt.head()


name,onset,offset,duration
str,i64,i64,i64
"""fixation""",2626913,2627069,156
"""fixation""",2627072,2627341,269
"""fixation""",2627345,2627640,295
"""fixation""",2629470,2629991,521
"""fixation""",2630014,2630296,282


In [45]:
saccades_ivt = gaze.events.frame.filter(pl.col("name") == "saccade")
saccades_ivt.head()

name,onset,offset,duration
str,i64,i64,i64


In [46]:
for th in [5, 10, 20, 30]:
    gaze.detect(method="ivt", velocity_threshold=th)
    print(th, gaze.events.frame["name"].unique())


5 shape: (4,)
Series: 'name' [str]
[
	"blink_eyelink"
	"saccade_eyelink"
	"fixation_eyelink"
	"fixation"
]
10 shape: (4,)
Series: 'name' [str]
[
	"blink_eyelink"
	"fixation"
	"saccade_eyelink"
	"fixation_eyelink"
]
20 shape: (4,)
Series: 'name' [str]
[
	"blink_eyelink"
	"saccade_eyelink"
	"fixation_eyelink"
	"fixation"
]
30 shape: (4,)
Series: 'name' [str]
[
	"fixation_eyelink"
	"saccade_eyelink"
	"blink_eyelink"
	"fixation"
]


In [47]:
for th in [5, 10, 20, 30]:
    gaze.detect(method="ivt", velocity_threshold=th)
    counts = gaze.events.frame["name"].value_counts()
    print(th, counts)


5 shape: (4, 2)
┌──────────────────┬───────┐
│ name             ┆ count │
│ ---              ┆ ---   │
│ str              ┆ u32   │
╞══════════════════╪═══════╡
│ fixation         ┆ 30558 │
│ saccade_eyelink  ┆ 16953 │
│ blink_eyelink    ┆ 579   │
│ fixation_eyelink ┆ 17047 │
└──────────────────┴───────┘
10 shape: (4, 2)
┌──────────────────┬───────┐
│ name             ┆ count │
│ ---              ┆ ---   │
│ str              ┆ u32   │
╞══════════════════╪═══════╡
│ fixation_eyelink ┆ 17047 │
│ saccade_eyelink  ┆ 16953 │
│ blink_eyelink    ┆ 579   │
│ fixation         ┆ 30558 │
└──────────────────┴───────┘
20 shape: (4, 2)
┌──────────────────┬───────┐
│ name             ┆ count │
│ ---              ┆ ---   │
│ str              ┆ u32   │
╞══════════════════╪═══════╡
│ fixation         ┆ 35520 │
│ saccade_eyelink  ┆ 16953 │
│ blink_eyelink    ┆ 579   │
│ fixation_eyelink ┆ 17047 │
└──────────────────┴───────┘
30 shape: (4, 2)
┌──────────────────┬───────┐
│ name             ┆ count │
│ ---

In [48]:
for th in [5, 10, 20, 30]:
    gaze.detect(method="ivt", velocity_threshold=th, clear=True)
    counts = gaze.events.frame["name"].value_counts()
    print(th, counts)


5 shape: (0, 2)
┌──────┬───────┐
│ name ┆ count │
│ ---  ┆ ---   │
│ str  ┆ u32   │
╞══════╪═══════╡
└──────┴───────┘
10 shape: (0, 2)
┌──────┬───────┐
│ name ┆ count │
│ ---  ┆ ---   │
│ str  ┆ u32   │
╞══════╪═══════╡
└──────┴───────┘
20 shape: (1, 2)
┌──────────┬───────┐
│ name     ┆ count │
│ ---      ┆ ---   │
│ str      ┆ u32   │
╞══════════╪═══════╡
│ fixation ┆ 4962  │
└──────────┴───────┘
30 shape: (1, 2)
┌──────────┬───────┐
│ name     ┆ count │
│ ---      ┆ ---   │
│ str      ┆ u32   │
╞══════════╪═══════╡
│ fixation ┆ 12798 │
└──────────┴───────┘


Step 1 pf peyepline: create the gaze frame.

	-- data collection folder
	---- ...
	---- fixations
	---- saccades(?)
	---- reading_measures
	---- raw_data (i.e. gaze sample csv)